<a href="https://colab.research.google.com/github/omarmorales/web_scrapping_python/blob/main/procesomx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install newspaper3k
import nltk
nltk.download('punkt')
from newspaper import Article
from newspaper import Config
import sys
sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time
from urllib.request import urlopen
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
wd.maximize_window()
wait = WebDriverWait(wd, 30)

  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
url = "https://www.proceso.com.mx/nacional/politica/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
r = requests.get(url, headers=headers)

wd.get(url)

for i in range(30):
  wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "vermasnoticias"))).click()

news = wd.find_elements(By.CLASS_NAME, "caja")
soup_l1 = soup(wd.page_source, 'lxml')
all_news = soup_l1.find_all("h2", {"class": "titulo"})


lnks = list(art.find('a') for art in all_news)


In [79]:
clean_lnks = list(url+(l.get('href')) for l in lnks)
len(clean_lnks)

450

In [80]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10

In [81]:
rows = []
for link in clean_lnks[:200]:
  a = Article(link)
  a.download()
  a.parse()
  a.nlp()

  title = a.title
  link = link
  text = a.summary
  publish_date = a.publish_date

  row = {
      'title': title,
      'link': link,
      'text': text,
      'published_on': publish_date
  }

  rows.append(row)

df = pd.DataFrame(rows)

In [82]:
df

,title,link,text,published_on
0,"García Vilchis denuncia ""nado sincronizado"" de...",https://www.proceso.com.mx/nacional/politica//...,Todos utilizan el mismo título que fue utiliza...,2022-02-23
1,"""Creyó que iba a ser cómplice y servil"": así r...",https://www.proceso.com.mx/nacional/politica//...,“Confirmo que el Presidente López Obrador come...,2022-02-22
2,"El Fisgón, Fernanda Tapia y Villamil defienden...",https://www.proceso.com.mx/nacional/politica//...,"El Fisgón y Jenaro Villamil, en el foro de la ...",2022-02-22
3,La Comisión de Bolsa y Valores de EU admitió d...,https://www.proceso.com.mx/nacional/politica//...,"""Ayer conocimos la versión 4T de Virgilio Andr...",2022-02-22
4,"Sheinbaum desafía al INE: ""¿Dónde quedó el árb...",https://www.proceso.com.mx/nacional/politica//...,Luego de que la semana pasada se enredó en dim...,2022-02-22
...,...,...,...,...
195,"En carta a AMLO, Lilly Téllez rechaza la visit...",https://www.proceso.com.mx/nacional/politica//...,CIUDAD DE MÉXICO (apro).-La senadora panista L...,2021-09-15
196,Invitación de AMLO a Quirino Ordaz es a título...,https://www.proceso.com.mx/nacional/politica//...,"“Es a título personal, nada tiene que ver con ...",2021-09-13
197,"AMLO arremente de nuevo contra el INE, ahora p...",https://www.proceso.com.mx/nacional/politica//...,Conteo de voto de mexicanos en el extranjero e...,2021-09-13
198,"El Tribunal Electoral sanciona a la sección ""Q...",https://www.proceso.com.mx/nacional/politica//...,García Vilchis y el presidente López Obrador e...,2021-09-09


In [83]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [84]:
df.to_csv('procesomx.csv')
!cp procesomx.csv "drive/My Drive/"